In [3]:
from glob import glob
import json

electra_outputs = glob('../archive/reviews_outputs/*')
tfidf_outputs = glob('../archive/reviews_tfidf/*')
print(len(electra_outputs), len(tfidf_outputs))

6574 7170


In [4]:
with open(tfidf_outputs[0]) as f:
    d1 = json.load(f)
    print(d1)

[['f00king', 0.26111177629066396], ['princess', 0.13509306314506886], ['pinkpine', 0.12036459785536878], ['f00k', 0.12036459785536878], ['ntr', 0.11572247750032481], ['luck;', 0.1042167343310379], ['h-manga', 0.09671278778012093], ['effort;', 0.09531321551929334], ['nympho', 0.0889237613737092], ['boyfriend', 0.08607021986071121], ['faps', 0.08386689402805328], ['preform', 0.0816081467421143], ['hentai', 0.080303421637608], ['nether', 0.07533000171779573], ['fapping', 0.06522406905208036], ['glossy', 0.06424444080772414], ['anal', 0.060636761977560724], ['maids', 0.05816700191374294], ['butler', 0.05275875860220402], ['fence', 0.0517452346613551], ['bust', 0.05026366251935546], ['busty', 0.050089170166372474], ['classified', 0.049467664746677596], ['partners', 0.048767708365128865], ['nuts', 0.047090557479988346], ['vanilla', 0.044597732552365356], ['reccomend', 0.04253067324021157], ['guys', 0.0424462006682553], ['overpowered', 0.04154576325884668], ['active', 0.039641796056857234], [

In [5]:
with open(electra_outputs[0]) as f:
    d2 = json.load(f)
    print(d2)

{'pred_scores': [0.90576171875, 0.99853515625, 0.96533203125], 'word_count': [['watch', 2], ['fapping', 2], ['ntr', 2], ['nether', 2], ['imo', 2], ['hentai', 1], ['reccomend', 1], ['borderline', 1], ['switch', 1], ['preform', 1], ['jealous', 1], ['bust', 1], ['animation', 1], ['vanilla', 1], ['anal', 1], ['faps', 1], ['active', 1], ['glossy', 1], ['putting', 1], ['exception', 1], ['luck', 1]]}


In [10]:
# 将electra的outpt归一化后合并到tfidf中去
tfidf_total = {}
from tqdm.notebook import tqdm
for t in tqdm(tfidf_outputs):
    name = t.split('/')[-1].split('.')[0]
    tfidf_total[name] = json.load(open(t))

HBox(children=(FloatProgress(value=0.0, max=7170.0), HTML(value='')))

In [11]:
for e in tqdm(electra_outputs):
    name = e.split('/')[-1].split('.')[0]
    if name not in tfidf_total:
        print('ERROR', name)
        continue
    # 在2个输出中均出现的，electra的输出增强tfidf，如果该单词出现在动画名中则忽略
    # 增强规则：min_max归一化后*1.5为倍率, [1+lambda(1.5)*min_max(electra)]*tfidf
    tfidf_words = {}
    for w in tfidf_total[name]:
        tfidf_words[w[0].lower()] = w[1]
    d = json.load(open(e))
    if len(d['word_count'])<2:
        continue
    max_count = d['word_count'][0][1]
    min_count = d['word_count'][-1][1]
    for w in d['word_count']:
        if w[0] not in tfidf_words or w[0] in name:
            continue
        boost = (w[1]-min_count+0.1)/(max_count-min_count+0.1)
        boost *= 1.5
        tfidf_words[w[0]] = tfidf_words[w[0]]+boost*tfidf_words[w[0]]
    tfidf_words = [[k, v] for k,v in tfidf_words.items()]
    tfidf_total[name] = tfidf_words

HBox(children=(FloatProgress(value=0.0, max=6574.0), HTML(value='')))

In [12]:
# 统计词频
import collections
word_count = collections.defaultdict(int)
for name in tqdm(tfidf_total):
    for w in tfidf_total[name]:
        word_count[w[0]]+=1

HBox(children=(FloatProgress(value=0.0, max=7170.0), HTML(value='')))

In [13]:
word_count = [(k, v) for k, v in word_count.items()]
word_count.sort(key=lambda x: x[1], reverse=True)
print(len(word_count))

427089


In [14]:
max_rev_toks = 70000
word_count = word_count[:max_rev_toks]

In [17]:
review_word_map = {}
for i, w in enumerate(word_count):
    review_word_map[w[0]] = i
with open('../archive/review_word_map.json','w') as w:
    json.dump(review_word_map, w)

In [20]:
filter_review_words = {}
for name in tqdm(tfidf_total):
    df = []
    for w in tfidf_total[name]:
        if w[0] not in review_word_map:
            continue
        df.append(w)
    df.sort(key=lambda x:x[1], reverse=True)
    filter_review_words[name] = df

HBox(children=(FloatProgress(value=0.0, max=7170.0), HTML(value='')))

In [21]:
with open('../archive/review_data.json','w') as w:
    json.dump(filter_review_words, w)

In [26]:
with open('../archive/reviews_tfidf/8291_Ojousama_wa_H_ga_Osuki__The_Animation.json') as f:
    d = json.load(f)

In [32]:
d = [w for w in d if w[0]!='f00king' and w[0]!='f00k']
d[:10]

[['princess', 0.13509306314506886],
 ['pinkpine', 0.12036459785536878],
 ['ntr', 0.11572247750032481],
 ['luck;', 0.1042167343310379],
 ['h-manga', 0.09671278778012093],
 ['effort;', 0.09531321551929334],
 ['nympho', 0.0889237613737092],
 ['boyfriend', 0.08607021986071121],
 ['faps', 0.08386689402805328],
 ['preform', 0.0816081467421143]]

In [33]:
filter_review_words['8291_Ojousama_wa_H_ga_Osuki__The_Animation'][:10]

[['ntr', 0.289306193750812],
 ['nether', 0.18832500429448934],
 ['fapping', 0.1630601726302009],
 ['princess', 0.13509306314506886],
 ['luck;', 0.1042167343310379],
 ['h-manga', 0.09671278778012093],
 ['effort;', 0.09531321551929334],
 ['faps', 0.09530328866824236],
 ['preform', 0.09273653038876624],
 ['watch', 0.09126315987354247]]